In [ ]:
# Correction Qiskit Tutorial
# Textbook: Learn Quantum Computation using Qiskit
# Title: Grover's Algorithm
# link: https://qiskit.org/textbook/ch-algorithms/grover.html#2.-Example:-2-Qubits-
# Section: "3.1 Qiskit Implementation"
# Capture: https://drive.google.com/file/d/13Di3TQ9PH_kta-Q4p6Vi8bdoXgIlNAMO/view?usp=sharing
# Date: 13Th September 2022

# Issue: When I try to implement the circuit I run into the next problem

# ---------------------------------------------------------------------------
# NameError                                 Traceback (most recent call last)
# <ipython-input-30-e3ba7032658b> in <module>
#      30 n = 3
#      31 grover_circuit = QuantumCircuit(n)
# ---> 32 grover_circuit = initialize_s(grover_circuit, [0,1,2])
#      33 grover_circuit.append(oracle_ex3, [0,1,2])
#      34 grover_circuit.append(diffuser(n), [0,1,2])

# NameError: name 'initialize_s' is not defined

# Following tutorial

In [30]:
from qiskit import *

qc = QuantumCircuit(3)
qc.cz(0, 2)
qc.cz(1, 2)
oracle_ex3 = qc.to_gate()
oracle_ex3.name = "U$_\omega$"

def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

n = 3
grover_circuit = QuantumCircuit(n)
grover_circuit = initialize_s(grover_circuit, [0,1,2])
grover_circuit.append(oracle_ex3, [0,1,2])
grover_circuit.append(diffuser(n), [0,1,2])
grover_circuit.measure_all()
grover_circuit.draw()

NameError: name 'initialize_s' is not defined

# Request

https://qiskit.org/textbook/ch-algorithms/images/grover_circuit_3qubits.png

# Z=HXH

In [102]:
n = 3

from qiskit.circuit.library import MCMTVChain

qc = QuantumCircuit(n)
qc.h(range(n-1, -1, -1))
qc.cz(2,0)
qc.cz(1,0)
qc.h(range(n-1, -1, -1))
qc.x(range(n-1, -1, -1))
qc.h(0)  # H on target qubit
qc.mcx([1,2], 0, mode='noancilla')
qc.h(0)  # again H on target qubit
qc.x(range(n-1, -1, -1))
qc.h(range(n-1, -1, -1))
qc.measure_all()
qc.draw()

┌───┐        ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐ ░ ┌─┐      
   q_0: ┤ H ├─■───■──┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─░─┤M├──────
        ├───┤ │   │  ├───┤├───┤└───┘└─┬─┘├───┤├───┤└───┘ ░ └╥┘┌─┐   
   q_1: ┤ H ├─┼───■──┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░──╫─┤M├───
        ├───┤ │ ┌───┐├───┤└───┘       │  ├───┤├───┤      ░  ║ └╥┘┌─┐
   q_2: ┤ H ├─■─┤ H ├┤ X ├────────────■──┤ X ├┤ H ├──────░──╫──╫─┤M├
        └───┘   └───┘└───┘               └───┘└───┘      ░  ║  ║ └╥┘
meas: 3/════════════════════════════════════════════════════╩══╩══╩═
                                                            0  1  2